# **Training Using Multilayer Perceptron and Genetic Algorithm**


# **EEG Brain Age Estimation**  
## PCA + Data Augmentation + Deep MLP (Optuna)
### Final Pipeline (2025)


This framework presents an end-to-end pipeline for **EEG-based Brain Age estimation**, combining signal preprocessing, dimensionality reduction, data augmentation, deep learning, and interpretability analysis.  
The goal is to predict **Brain Age** from EEG features and analyze the **Brain Age Gap (BAG)** as a biomarker of neural aging.

**Brain Age Gap (BAG)** is defined as:

\begin{equation}

\text{BAG} = \text{Predicted Brain Age} - \text{Chronological Age}

\end{equation}
A positive BAG may indicate accelerated brain aging, while a negative BAG may suggest preserved or delayed aging.

- EEG → PSD → Relative Band Power → Log Transform  
- PCA for dimensionality reduction  
- Data augmentation for robustness  
- Deep MLP optimized with Genetic Algorithm
- BAG analysis for neuroaging interpretation  

This framework provides a **robust, interpretable, and publication-ready approach** to EEG-based Brain Age estimation in 2025.


### **Librerias**

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:
!pip install deap

In [10]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance
from tensorflow.keras import layers, models, callbacks, optimizers
import tensorflow as tf
import optuna
import warnings
import random
import numpy as np
from deap import base, creator, tools



In [11]:
warnings.filterwarnings("ignore")

# Reproducibilidad
np.random.seed(42)
tf.keras.utils.set_random_seed(42)

### **Load EEG Dataset**

In [34]:
DATA_PATH = r"C:\Users\Ale\Downloads\Jade\Ale\Data_Parametrizada\EEG_features_final.csv"
SAVE_DIR = r"C:\Users\Ale\Downloads\Jade\Ale\Jade_Saves\Results_MLP_Genetic"
os.makedirs(SAVE_DIR, exist_ok=True)

In [5]:
df = pd.read_csv(DATA_PATH)
df = df.select_dtypes(include=[np.number])
X = df.drop(columns=["Age"])
y = df["Age"]

print(f" Loaded {len(X)} samples, {X.shape[1]} features.")


 Loaded 510 samples, 9 features.


### **Cleaning and PCA**

In [6]:
z = np.abs(stats.zscore(X))
mask = (z < 3).all(axis=1)
X, y = X[mask], y[mask]

##Standardization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#PCA
pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(X_scaled)
print(f" PCA reduced features to {X_pca.shape[1]} components.\n")

 PCA reduced features to 4 components.



### **Synthetic data augmentation**

In [7]:
X_aug, y_aug = X_pca.copy(), y.copy()
for _ in range(3):
    noise = np.random.normal(0, 0.01, X_pca.shape)
    X_aug = np.vstack([X_aug, X_pca + noise])
    y_aug = pd.concat([y_aug, y], ignore_index=True)

print(f" Augmented dataset: {X_aug.shape[0]} samples.\n")

print("Splitting data into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(X_aug, y_aug, test_size=0.2, random_state=42)
print(f" Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}\n")


 Augmented dataset: 1968 samples.

Splitting data into train and test sets...
 Training samples: 1574, Testing samples: 394



## **MLP model for Optuna optimization**

### Funciones Auxiliares

In [ ]:
# ===================== FITNESS =====================
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# ===================== INDIVIDUAL =====================
def random_individual():
    n_layers = random.randint(2, 4)   # reduced for speed
    lr = 10 ** random.uniform(-4, -2)
    batch_size = random.choice([16, 32, 48])
    units = [random.randint(64, 256) for _ in range(n_layers)]
    drops = [random.uniform(0.1, 0.3) for _ in range(n_layers)]
    return creator.Individual([n_layers, lr, batch_size, units, drops])

# ===================== REPAIR =====================
def repair_individual(individual):
    n_layers, lr, batch_size, units, drops = individual
    units = units[:n_layers]
    drops = drops[:n_layers]
    while len(units) < n_layers:
        units.append(random.randint(64, 256))
    while len(drops) < n_layers:
        drops.append(random.uniform(0.1, 0.3))
    individual[:] = [n_layers, lr, batch_size, units, drops]
    return individual

# ===================== BUILD MODEL =====================
def build_ga_mlp(individual, input_dim):
    n_layers, lr, _, units, drops = individual
    model = models.Sequential([layers.Input(shape=(input_dim,))])
    for i in range(n_layers):
        model.add(layers.Dense(units[i], activation="relu"))
        model.add(layers.Dropout(drops[i]))
    model.add(layers.Dense(1, activation="linear"))
    model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss="mae")
    return model

# ===================== EVALUATE =====================
def evaluate(individual):
    repair_individual(individual)

    print(f"    ▶ Training MLP | Layers={individual[0]} | LR={individual[1]:.1e} | BS={individual[2]}")

    model = build_ga_mlp(individual, X_train.shape[1])

    es = callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True
    )

    model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=60,
        batch_size=individual[2],
        verbose=1,              
        callbacks=[es]
    )

    y_pred = model.predict(X_test, verbose=0).flatten()
    r2 = r2_score(y_test, y_pred)

    print(f"    ✔ Finished | R² = {r2:.4f}\n")
    return (r2,)

# ===================== MUTATION =====================
def mutate_individual(individual, indpb=0.2):
    n_layers, lr, batch_size, units, drops = individual

    if random.random() < indpb:
        n_layers = random.randint(2, 4)

    if random.random() < indpb:
        lr = 10 ** random.uniform(-4, -2)

    if random.random() < indpb:
        batch_size = random.choice([16, 32, 48])

    for i in range(len(units)):
        if random.random() < indpb:
            units[i] = random.randint(64, 256)

    for i in range(len(drops)):
        if random.random() < indpb:
            drops[i] = random.uniform(0.1, 0.3)

    individual[:] = [n_layers, lr, batch_size, units, drops]
    repair_individual(individual)
    return (individual,)

### Toolbox

In [ ]:
toolbox = base.Toolbox()
toolbox.register("individual", random_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate_individual)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
# ===================== GA RUN =====================
POP_SIZE = 10
N_GEN = 5
CX_PROB = 0.7
MUT_PROB = 0.2

population = toolbox.population(n=POP_SIZE)

print("\n🔹 Evaluating initial population...\n")
for i, ind in enumerate(population):
    print(f"  Individual {i+1}/{POP_SIZE}")
    ind.fitness.values = toolbox.evaluate(ind)

for gen in range(N_GEN):
    print(f"\n================ GENERATION {gen + 1}/{N_GEN} ================")

    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    for c1, c2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CX_PROB:
            toolbox.mate(c1, c2)
            repair_individual(c1)
            repair_individual(c2)
            del c1.fitness.values, c2.fitness.values

    for mutant in offspring:
        if random.random() < MUT_PROB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    invalid = [ind for ind in offspring if not ind.fitness.valid]
    print(f" Re-evaluating {len(invalid)} offspring...\n")

    for ind in invalid:
        ind.fitness.values = toolbox.evaluate(ind)

    population[:] = offspring

# ===================== BEST RESULT =====================
best_ind = tools.selBest(population, 1)[0]

print("\n================ BEST MODEL FOUND ================")
print("Layers:", best_ind[0])
print("Learning rate:", f"{best_ind[1]:.2e}")
print("Batch size:", best_ind[2])
print("Units:", best_ind[3])
print("Dropouts:", best_ind[4])
print("Best R²:", best_ind.fitness.values[0])




🔹 Evaluating initial population...

  Individual 1/10
    ▶ Training MLP | Layers=2 | LR=5.6e-04 | BS=32
Epoch 1/60
40/40 [==============================] - 0s 4ms/step - loss: 47.8350 - val_loss: 43.6535
Epoch 2/60
40/40 [==============================] - 0s 3ms/step - loss: 35.9840 - val_loss: 26.5037
Epoch 3/60
40/40 [==============================] - 0s 2ms/step - loss: 22.0891 - val_loss: 19.8773
Epoch 4/60
40/40 [==============================] - 0s 2ms/step - loss: 19.5503 - val_loss: 19.3829
Epoch 5/60
40/40 [==============================] - 0s 3ms/step - loss: 18.9168 - val_loss: 18.9420
Epoch 6/60
40/40 [==============================] - 0s 3ms/step - loss: 18.4998 - val_loss: 18.6046
Epoch 7/60
40/40 [==============================] - 0s 2ms/step - loss: 18.2826 - val_loss: 18.3828
Epoch 8/60
40/40 [==============================] - 0s 3ms/step - loss: 17.8746 - val_loss: 18.1180
Epoch 9/60
40/40 [==============================] - 0s 3ms/step - loss: 17.5005 - val_loss: 17

### Best Parameters

In [ ]:
best_params = {
    "n_layers": best_ind[0],
    "learning_rate": best_ind[1],
    "batch_size": best_ind[2],
    "units": best_ind[3],
    "dropouts": best_ind[4]
}

print("\nTraining final model with best GA hyperparameters...")
print(best_params)



Training final model with best GA hyperparameters...
{'n_layers': 4, 'learning_rate': 0.004992982180425417, 'batch_size': 16, 'units': [231, 253, 72, 123], 'dropouts': [0.13202714224307185, 0.1758528044479632, 0.21054028790593904, 0.259551157660853]}


### **Final training with best parameters**

In [ ]:
def build_final_ga_mlp(best_params, input_dim):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_dim,)))

    for i in range(best_params["n_layers"]):
        model.add(layers.Dense(
            best_params["units"][i],
            activation="relu"
        ))
        model.add(layers.Dropout(
            best_params["dropouts"][i]
        ))

    model.add(layers.Dense(1, activation="linear"))

    model.compile(
        optimizer=optimizers.Adam(
            learning_rate=best_params["learning_rate"]
        ),
        loss="mae"
    )
    return model


In [30]:
final_model = build_final_ga_mlp(best_params, X_train.shape[1])

es = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=25,
    restore_best_weights=True
)

rlrop = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=10
)

final_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=400,              # full training
    batch_size=best_params["batch_size"],
    verbose=1,
    callbacks=[es, rlrop]
)


Epoch 1/400
79/79 [==============================] - 1s 4ms/step - loss: 23.4279 - val_loss: 19.5332 - lr: 0.0050
Epoch 2/400
79/79 [==============================] - 0s 4ms/step - loss: 18.6375 - val_loss: 16.9289 - lr: 0.0050
Epoch 3/400
79/79 [==============================] - 0s 3ms/step - loss: 18.3166 - val_loss: 16.6608 - lr: 0.0050
Epoch 4/400
79/79 [==============================] - 0s 3ms/step - loss: 16.6956 - val_loss: 15.7407 - lr: 0.0050
Epoch 5/400
79/79 [==============================] - 0s 3ms/step - loss: 16.0678 - val_loss: 15.1486 - lr: 0.0050
Epoch 6/400
79/79 [==============================] - 0s 3ms/step - loss: 15.6414 - val_loss: 15.8935 - lr: 0.0050
Epoch 7/400
79/79 [==============================] - 0s 3ms/step - loss: 15.6216 - val_loss: 16.7088 - lr: 0.0050
Epoch 8/400
79/79 [==============================] - 0s 3ms/step - loss: 16.1854 - val_loss: 14.9240 - lr: 0.0050
Epoch 9/400
79/79 [==============================] - 0s 3ms/step - loss: 14.5185 - val_l

In [31]:
y_pred = final_model.predict(X_test, verbose=0).flatten()

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
r = np.corrcoef(y_test, y_pred)[0, 1]

print("\n================ FINAL TEST RESULTS (GA) =================")
print(f"MAE (years): {mae:.2f}")
print(f"R²: {r2:.3f}")
print(f"Pearson r: {r:.3f}")
print("========================================================")



================ FINAL TEST RESULTS (GA) =================
MAE (years): 1.51
R²: 0.988
Pearson r: 0.994


### **BAG Calculation and Categorization**

In [32]:
def categorize_bag(bag):
    if bag < -3: return "Resilient"
    elif bag > 3: return "Accelerated"
    else: return "Normal"

In [35]:
BAG = y_pred - y_test.values
bag_mean, bag_std = BAG.mean(), BAG.std()
print(f"\n Brain Age Gap (BAG): mean={bag_mean:.2f}, std={bag_std:.2f}")


bag_categories = np.vectorize(categorize_bag)(BAG)
bag_df = pd.DataFrame({
    "Chronological_Age": y_test.values,
    "Predicted_Age": y_pred,
    "BAG": BAG,
    "Category": bag_categories
})
bag_df.to_csv(os.path.join(SAVE_DIR, "EEG_Brain_Age_Gap_MLP.csv"), index=False)



 Brain Age Gap (BAG): mean=-0.16, std=2.12


In [36]:
# Histogram
plt.figure(figsize=(7,5))
plt.hist(BAG, bins=25, color="skyblue", edgecolor="black")
plt.axvline(0, color="red", linestyle="--")
plt.title("Brain Age Gap (Predicted - Chronological)")
plt.xlabel("BAG (years)")
plt.ylabel("Count")
plt.grid(alpha=0.4)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "1.Bag_hist_mlp.png"))
plt.close()


In [37]:
# Category barplot
plt.figure(figsize=(6,4))
pd.Series(bag_categories).value_counts().reindex(["Resilient","Normal","Accelerated"]).plot(kind="bar", color=["#66c2a5","#fc8d62","#8da0cb"])
plt.title("BAG Categories Distribution (±3 years)")
plt.ylabel("Count")
plt.grid(axis="y", alpha=0.4)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "2.Bag_categories_mlp.png"))
plt.close()

### **Permutation Importance**

In [38]:
print("\n Computing permutation importance...")
X_test_df = pd.DataFrame(X_test, columns=[f"PC{i+1}" for i in range(X_test.shape[1])])
perm_result = permutation_importance(
    final_model, X_test_df, y_test, n_repeats=10, random_state=42, scoring="r2"
)
perm_df = pd.DataFrame({
    "Feature": X_test_df.columns,
    "Importance": perm_result.importances_mean
}).sort_values("Importance", ascending=False)

perm_df.to_csv(os.path.join(SAVE_DIR, "MLP_feature_importance.csv"), index=False)
plt.figure(figsize=(8,6))
plt.barh(perm_df["Feature"][:10], perm_df["Importance"][:10], color="cornflowerblue")
plt.gca().invert_yaxis()
plt.title("Top 10 Feature Importances (Permutation Importance – MLP)")
plt.xlabel("Mean R² Decrease")
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "mlp_permutation_importance.png"))
plt.close()



 Computing permutation importance...
13/13 [==============================] - 0s 1ms/step


In [39]:
final_model.save(os.path.join(SAVE_DIR, "MLP_Deep_Optuna_FINAL.keras"))
pd.DataFrame([{
    "MAE": mae, "R2": r2, "r": r, "BAG_mean": bag_mean, "BAG_std": bag_std,
    "Best_Params": best_params, "PCA_Components": X_pca.shape[1]
}]).to_csv(os.path.join(SAVE_DIR, "MLP_Deep_Optuna_results.csv"), index=False)

print(f"\n All results saved in: {SAVE_DIR}")


 All results saved in: C:\Users\Ale\Downloads\Jade\Ale\Jade_Saves\Results_MLP_Genetic


Due to the stochastic nature of multilayer perceptron training, the final model was trained independently 20 times using the same dataset and optimal hyperparameters. Performance metrics from each run were recorded to assess model robustness and stability, and results are reported as mean and standard deviation.

In [40]:
def BAG_stadistics(y_pred, y_test):
    BAG = y_pred - y_test.values
    bag_mean, bag_std = BAG.mean(), BAG.std()
    print(f"\n Brain Age Gap (BAG): mean={bag_mean:.2f}, std={bag_std:.2f}")
    return bag_mean, bag_std

    
def comparison_trials(n_trials, best_params, X_train, y_train, X_test, y_test):
    results = []
    for i in range(n_trials):
        print("\nTraining final model with best hyperparameters...")
        print("Trial:", i+1)
        final_model = build_final_ga_mlp(best_params, X_train.shape[1])

        es = callbacks.EarlyStopping(
            monitor="val_loss",
            patience=25,
            restore_best_weights=True
        )

        rlrop = callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=10
        )

        final_model.fit(
            X_train,
            y_train,
            validation_split=0.2,
            epochs=400,              # full training
            batch_size=best_params["batch_size"],
            verbose=1,
            callbacks=[es, rlrop]
        )

     
        y_pred = final_model.predict(X_test, verbose=0).flatten()

        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        r = np.corrcoef(y_test, y_pred)[0, 1]


        results.append({
            "Trial": i + 1,
            "MAE": mae,
            "R2": r2,
            "Pearson_r": r,
            "BAG_mean": bag_mean,
            "BAG_std": bag_std,
            "Batch_size": best_params["batch_size"],
            "PCA_Components": X_pca.shape[1]
        })

    # save once at the end
    results_df = pd.DataFrame(results)
    results_df.to_csv(
        os.path.join(SAVE_DIR, "TRIALS_MLP_Deep_Optuna_Genetic.csv"),
        index=False
    )

    print(f"\nAll results saved in: {SAVE_DIR}")



In [42]:
import time
star_time = time.time()
n=20
comparison_trials(20, best_params, X_train, y_train, X_test, y_test)
end_time = time.time()
print(f"\n Total comparison trials time: {end_time - star_time:.2f} seconds") 


Training final model with best hyperparameters...
Trial: 1
Epoch 1/400
79/79 [==============================] - 1s 6ms/step - loss: 23.3233 - val_loss: 17.8550 - lr: 0.0050
Epoch 2/400
79/79 [==============================] - 0s 3ms/step - loss: 18.4664 - val_loss: 18.6127 - lr: 0.0050
Epoch 3/400
79/79 [==============================] - 0s 3ms/step - loss: 18.2716 - val_loss: 16.6403 - lr: 0.0050
Epoch 4/400
79/79 [==============================] - 0s 3ms/step - loss: 17.3857 - val_loss: 16.2425 - lr: 0.0050
Epoch 5/400
79/79 [==============================] - 0s 3ms/step - loss: 16.4793 - val_loss: 15.2541 - lr: 0.0050
Epoch 6/400
79/79 [==============================] - 0s 3ms/step - loss: 15.8573 - val_loss: 17.0689 - lr: 0.0050
Epoch 7/400
79/79 [==============================] - 0s 3ms/step - loss: 16.1381 - val_loss: 14.3865 - lr: 0.0050
Epoch 8/400
79/79 [==============================] - 0s 3ms/step - loss: 15.4129 - val_loss: 12.7948 - lr: 0.0050
Epoch 9/400
79/79 [=========

Trials time = 1497.91 seconds